In [272]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [422]:
import random as ran
from IPython.display import clear_output
from src import spotify, dp, utils, DB
import pandas as pd

db = DB()

In [423]:
genres_cat = ['k-pop', 'pop', 'hip-hop', 'ballad', 'jazz', "test"]

genres_dict = dict({
    "k-pop": "3dbLT62Cvs46Ju7a8gpr36",
    "pop": "3gzIK2dGSVvPBiohsTwEWZ",
    "hip-hop": "2FWquqPNxte8iqZ3ATQG0p",
    "ballad": "3UPjb91Fwm7u2tAm92Bk0p",
    "jazz": "7tIOwGFRTM08JdAeKOELGp",
    "test": "22skzmqfdWrjJylampe0kt"
})

In [424]:
mail_box_seed_list = list()

for _ in range(0, 10):
    genre_cnt = ran.randrange(1, 4)
    genre_cat = list()
    
    while len(genre_cat) != genre_cnt:
        ran_genres_cat = genres_cat[
            ran.randrange(0, len(genres_cat))
        ]
        
        if ran_genres_cat not in genre_cat:
            genre_cat.append(ran_genres_cat)

    mail_box_seed_list.append(genre_cat)
    
mail_box_seed_list

[['hip-hop', 'pop', 'test'],
 ['jazz', 'k-pop', 'test'],
 ['pop'],
 ['hip-hop'],
 ['k-pop'],
 ['jazz', 'pop'],
 ['jazz', 'k-pop'],
 ['k-pop'],
 ['pop'],
 ['k-pop', 'hip-hop']]

In [425]:
# 1. Get Token
token = spotify.get_token()

mail_boxes_list = list()
sel_track_list = list()
sel_tracks = pd.DataFrame()

for box_seed in mail_box_seed_list:
    sel_tracks = pd.DataFrame()
    
    for seed_genre in box_seed:
        seed_id = genres_dict[seed_genre]
        
        sel_tracks = sel_tracks.append(spotify.direct_sel(token, seed_id), ignore_index=True)
    
    sel_track_list.append(sel_tracks)
    mail_box_id = str(db.regist_mail_box(sel_tracks).inserted_id)
    
    mail_boxes_list.append(mail_box_id)

print(mail_boxes_list)
sel_track_list[0]

['623f2abbff1c441a392ab4c3', '623f2abbff1c441a392ab4c4', '623f2abbff1c441a392ab4c5', '623f2abbff1c441a392ab4c6', '623f2abbff1c441a392ab4c7', '623f2abbff1c441a392ab4c8', '623f2abcff1c441a392ab4c9', '623f2abcff1c441a392ab4ca', '623f2abcff1c441a392ab4cb', '623f2abcff1c441a392ab4cc']


,id,name,artists,artists_name
0,2FWquqPNxte8iqZ3ATQG0p,All I Wanna Do (K) (Feat. Hoody & Loco),4XDi67ZENZcbfKnvMnTYsI,Jay Park
1,3gzIK2dGSVvPBiohsTwEWZ,Easy,3WGpXCj9YhhfX11TToZcXP,Troye Sivan
2,22skzmqfdWrjJylampe0kt,Can't Hold Us (feat. Ray Dalton),3JhNCzhSMTxs9WLGJJxWOY,Macklemore


In [426]:
features_list = list()
genre_seed_list = list()
for sel_track in sel_track_list:
    genres = spotify.get_genres(token, sel_track)
    genre_seed_list.append(genres)
    
    features = spotify.get_features(token, sel_track)
    features_list.append(features)
    
    db.observe_seed_zone(features)
    
print(genre_seed_list[9])
features_list[9]

                  artists             genres
0  6YVMFz59CuY7ngCxTxjpxE          k-pop,pop
1  4XDi67ZENZcbfKnvMnTYsI  k-pop,pop,hip-hop


,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3dbLT62Cvs46Ju7a8gpr36,0.727,0.879,9,-1.167,0.129,0.124,0.000012,0.2440,0.671,146.959
1,2FWquqPNxte8iqZ3ATQG0p,0.403,0.724,8,-6.064,0.246,0.377,0.000000,0.0927,0.428,97.133


In [427]:
reco_list = list()
reco_features_list = list()

for idx, sel_track in enumerate(sel_track_list):
    print(idx)
    features = features_list[idx]
    genres = genre_seed_list[idx]
    
    recos = spotify.get_recommend(sel_track, features, genres, token)
    reco_features = spotify.get_features(token, recos)
    
    reco_list.append(recos)
    reco_features_list.append(reco_features)

0
1
2
3
4
5
6
7
8
9


In [428]:
reco_list[0].head()

,id,name,artists,artists_name
0,2PcJoVkfjcbvoqBxV1Nj7y,All I Wanna Do,4XDi67ZENZcbfKnvMnTYsI,Jay Park
1,4Cdyx3bs8mMYVJWP3wHPXj,On Rainy Days,1Pr9gT0veB2tgcisQeIGoC,Beast
2,2xYnqYBNL62ypPiA0wjD0s,AUTOMATIC,0LyfQWJT6nXafLPZqxe9Of,Various Artists
3,6mLfR7fHVDcCKpLwGLWCWw,F With U (feat. Ty Dolla $ign),6KZDXtSj0SzGOV705nNeh3,Kid Ink
4,2IgbYlOlFpiSFYnsqB39lM,Jasmine,0siBQaURCli5wn2lqv8WZg,DPR LIVE


In [429]:
reco_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2PcJoVkfjcbvoqBxV1Nj7y,0.613,0.668,8,-6.867,0.2670,0.245,0.0,0.0735,0.509,96.176
1,4Cdyx3bs8mMYVJWP3wHPXj,0.688,0.612,8,-6.303,0.0574,0.720,0.0,0.1060,0.692,90.022
2,2xYnqYBNL62ypPiA0wjD0s,0.720,0.703,8,-5.281,0.0409,0.133,0.0,0.1210,0.492,99.999
3,6mLfR7fHVDcCKpLwGLWCWw,0.578,0.528,8,-7.169,0.2740,0.400,0.0,0.1490,0.622,98.555
4,2IgbYlOlFpiSFYnsqB39lM,0.597,0.486,8,-5.893,0.0986,0.331,0.0,0.1400,0.397,137.680


In [430]:
norm_features_list = list()

for idx, features in enumerate(features_list):
    reco_features = reco_features_list[idx]
    norm_features = dp.make_norm(features, reco_features)
    norm_features_list.append(norm_features)
    
norm_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2FWquqPNxte8iqZ3ATQG0p,0.197138,0.632308,0.727273,0.535374,0.502545,0.471239,0.000000,0.109623,0.378280,0.207348
1,3gzIK2dGSVvPBiohsTwEWZ,0.572337,0.393846,0.000000,0.395784,0.046043,0.072605,0.001037,0.133179,0.592745,0.620710
2,22skzmqfdWrjJylampe0kt,0.562798,0.944615,0.181818,0.725465,0.127487,0.033354,0.000000,0.118707,0.893908,0.648939
3,2PcJoVkfjcbvoqBxV1Nj7y,0.531002,0.546154,0.727273,0.439733,0.551134,0.306235,0.000000,0.080062,0.470682,0.198718
4,4Cdyx3bs8mMYVJWP3wHPXj,0.650238,0.460000,0.727273,0.506908,0.066173,0.899998,0.000000,0.130100,0.679443,0.143217


In [431]:
kmeans_list = list()

for idx, norm_features in enumerate(norm_features_list):
    print("-- 라운드 {}".format(idx + 1))
    kmeans = utils.KMeans(
        datas=norm_features
    )
    kmeans.run(early_stop_cnt=3)
    clear_output(wait=True)
    
    kmeans_list.append(kmeans)

-- 라운드 10
1 round : ECV 35.58%
2 round : ECV 50.65%
3 round : ECV 52.75%
4 round : ECV 53.75%
5 round : ECV 54.28%
6 round : ECV 54.45%
7 round : ECV 54.52%
8 round : ECV 54.52%
9 round : ECV 54.52%
10 round : ECV 54.52%
11 round : ECV 54.52%
Clustering End.


In [432]:
filterings = list()
recos_list = list()

for idx, kmeans in enumerate(kmeans_list):
    sel_track = sel_track_list[idx]
    recos = reco_list[idx]
    
    _filtering_music_list = dp.music_filtering(sel_track, kmeans)
    filterings.append(_filtering_music_list)
    
    reco_musics = [_ in _filtering_music_list for _ in recos['id']]
    recos_list.append(recos[reco_musics])
    
recos_list[0].head()

,id,name,artists,artists_name
1,4Cdyx3bs8mMYVJWP3wHPXj,On Rainy Days,1Pr9gT0veB2tgcisQeIGoC,Beast
4,2IgbYlOlFpiSFYnsqB39lM,Jasmine,0siBQaURCli5wn2lqv8WZg,DPR LIVE
6,4AtDiqnMxDt8crflZWgZG5,Turn Off Your Phone (Feat. ELO) - Remix,4XDi67ZENZcbfKnvMnTYsI,Jay Park
7,6GS3lnAVy5w6AHWEKYzYeS,What 2 Do,3eCd0TZrBPm2n9cDG6yWfF,DEAN
17,6z1kLsntE7FuzKZHZWrXYN,instagram,3eCd0TZrBPm2n9cDG6yWfF,DEAN


In [433]:
for idx,_mail in enumerate(recos_list):
    mail_box_id = mail_boxes_list[idx]
    
    res = db.regist_mail(mail_box_id, _mail)
    mail_id = res.inserted_id
    
    print("우체통({})에\n추천 플레이리스트 MuLetter({})가 등록되었습니다.".format(
        mail_box_id
        ,str(mail_id)))

우체통(623f2abbff1c441a392ab4c3)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4cd)가 등록되었습니다.
우체통(623f2abbff1c441a392ab4c4)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4ce)가 등록되었습니다.
우체통(623f2abbff1c441a392ab4c5)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4cf)가 등록되었습니다.
우체통(623f2abbff1c441a392ab4c6)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4d0)가 등록되었습니다.
우체통(623f2abbff1c441a392ab4c7)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4d1)가 등록되었습니다.
우체통(623f2abbff1c441a392ab4c8)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4d2)가 등록되었습니다.
우체통(623f2abcff1c441a392ab4c9)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4d3)가 등록되었습니다.
우체통(623f2abcff1c441a392ab4ca)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4d4)가 등록되었습니다.
우체통(623f2abcff1c441a392ab4cb)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4d5)가 등록되었습니다.
우체통(623f2abcff1c441a392ab4cc)에
추천 플레이리스트 MuLetter(623f2accff1c441a392ab4d6)가 등록되었습니다.


### reply

In [434]:
import math as mt

In [435]:
track_list = list()
reco_track_list = list()

for mail_box_id in mail_boxes_list:
    tracks = db.get_mail_box(mail_box_id)
    reco_tracks = db.get_mail(mail_box_id)

    track_list.append(tracks)
    reco_track_list.append(reco_tracks)
    
track_list[0].head()

,id,name,artists,artists_name
0,2FWquqPNxte8iqZ3ATQG0p,All I Wanna Do (K) (Feat. Hoody & Loco),4XDi67ZENZcbfKnvMnTYsI,Jay Park
1,3gzIK2dGSVvPBiohsTwEWZ,Easy,3WGpXCj9YhhfX11TToZcXP,Troye Sivan
2,22skzmqfdWrjJylampe0kt,Can't Hold Us (feat. Ray Dalton),3JhNCzhSMTxs9WLGJJxWOY,Macklemore


In [436]:
sel_track_list = list()

for idx, reco_tracks in enumerate(reco_track_list):
    mail_box_id = mail_boxes_list[idx]
    n = mt.floor(len(reco_tracks) * 0.075)
    n = 2 if n <= 1 else n
    
    sel_tracks = reco_tracks.sample(n=n)
    db.new_mail_box_seed(mail_box_id, sel_tracks)

    sel_track_list.append(sel_tracks)

In [439]:
# 1. Get Token
token = spotify.get_token()

_sel_track_list = list()
features_list = list()
recos_list = list()
reco_features_list = list()

for idx, sel_tracks in enumerate(sel_track_list):
    print(idx)
    tracks = track_list[idx]
    _sel_tracks = pd.concat([tracks,sel_tracks], ignore_index=True)
    genres = spotify.get_genres(token, _sel_tracks)

    og_features = spotify.get_features(token, tracks)
    reply_features = spotify.get_features(token, sel_tracks)
    db.observe_seed_zone(reply_features)

    recos = spotify.get_recommend(sel_tracks, reply_features, genres, token,
                                 og=tracks)
    
    if str(type(recos)) == "<class 'requests.models.Response'>":
        print(recos.url)
        print(idx,recos.json())
        
    reco_features = spotify.get_features(token, recos)
    features = pd.concat([og_features, reply_features], ignore_index=True)
    
    features_list.append(features)
    recos_list.append(recos)
    reco_features_list.append(reco_features)
    _sel_track_list.append(_sel_tracks)

0
1
2
3
4
5
6
7
8
9


In [440]:
norm_features_list = list()

for idx, reco_features in enumerate(reco_features_list):
    features = features_list[idx]
    norm_features = dp.make_norm(features, reco_features)

    norm_features_list.append(norm_features)

In [441]:
filterings = list()
kmeans_list = list()

for idx, sel_tracks in enumerate(sel_track_list):
    print("{}번 박스".format(idx))
    norm_features = norm_features_list[idx]
    kmeans = utils.KMeans(
        datas=norm_features
    )
    kmeans.run(early_stop_cnt=5)
    clear_output(wait=True)

    _filtering_music_list = dp.music_filtering(sel_tracks, kmeans)

    if len(_filtering_music_list) <= (100 + len(sel_tracks)):
        filterings.append(_filtering_music_list)
        kmeans_list.append(kmeans)
        continue
    else:
        filter_music = norm_features.set_index("id").loc[_filtering_music_list].reset_index()
    while True:
        kmeans = utils.KMeans(
            datas=filter_music
        )
        kmeans.run(early_stop_cnt=5)
        clear_output(wait=True)


        _filtering_music_list = dp.music_filtering(sel_tracks, kmeans)

        if len(_filtering_music_list) <= (100 + len(sel_tracks)):
            filterings.append(_filtering_music_list)
            kmeans_list.append(kmeans)
            break
        else:
            filter_music = norm_features.set_index("id").loc[_filtering_music_list].reset_index()

9번 박스
1 round : ECV 27.96%
2 round : ECV 46.2%
3 round : ECV 48.86%
4 round : ECV 49.87%
5 round : ECV 50.13%
6 round : ECV 50.25%
7 round : ECV 50.5%
8 round : ECV 52.06%
9 round : ECV 53.54%
10 round : ECV 54.19%
11 round : ECV 55.05%
12 round : ECV 55.71%
13 round : ECV 55.99%
14 round : ECV 56.01%
15 round : ECV 56.02%
16 round : ECV 56.02%
17 round : ECV 56.02%
18 round : ECV 56.02%
19 round : ECV 56.02%
20 round : ECV 56.02%
21 round : ECV 56.02%
Clustering End.


In [442]:
reco_musics = [_ in filterings[1] for _ in recos_list[1]['id']]
recos_list[1][reco_musics]

,id,name,artists,artists_name
0,0TWducu0FOa75QjURpxPVn,Sunrise of the Planetary Dream Collector,"7DnLQaXsqcYkgm0nyDrB3r,0M6xcJTswOl2qvExCJhiS3","Terry Riley,Kronos Quartet"
1,5HfVLuge7NXEKY2ErYWsVN,Oboe Concerto: Oboe Concerto: III. Allegro,"2B7g3x3ODIEjpdoOLIoa2C,3P1VtkpIYbw6YoTo0KYlxy,...","Nicholas Daniel,Britten Sinfonia,James MacMillan"
3,3QaPlDueCFnbJdPCeK0RbM,Guillaume Tell: Guillaume Tell: Overture,"0roWUeP7Ac4yK4VN6L2gF4,2rslujXEON2msrHEXH3VjL,...","Gioachino Rossini,Prague Sinfonia,Christian Benda"
11,4GZs1nmdkgT0oaxQy7rugq,Never Let Me Go,375zxMmh2cSgUzFFnva0O7,Wynton Marsalis
12,4N6JkrGD78eLooaTDmwo9s,Weaver Of Dreams - Remastered 2003/Rudy Van Ge...,0fTHKjepK5HWOrb2rkS5Em,Freddie Hubbard
...,...,...,...,...
624,7kTlHVCARylJAqm40yVoWE,Stay Next To Me (with Chelsea Cutler),3ApUX1o6oSz321MMECyIYd,Quinn XCII
643,6wWaVoUOzLQJHd3bWAUpdZ,"Never Recover (Lil Baby & Gunna, Drake)","5f7VJjfbwm532GiveGC0ZK,2hlmm7s2ICUX0LVIhVFlZQ","Lil Baby,Gunna"
656,6SI4JD7iyQ0rrudhCNtMv0,"Ice Cream (feat. Ghostface Killah, Method Man ...",2yQf6b8hxahZaT5dHlWaB1,Raekwon
686,1aMX40ZlCtuYnAIdNLR1jH,Coming of Age,3TOKZohBEXnd5RgCXHUsH3,E-Dubble


In [443]:
for idx,mail_box_id in enumerate(mail_boxes_list):
    recos = recos_list[idx]
    filtering = filterings[idx]
    reco_musics = [_ in filtering for _ in recos['id']]
    
    db.new_mail(mail_box_id,
               recos[reco_musics])